Non-bayesian

In [1]:
package_paths = [r"C:\Users\benja\Documents\projects\goalscorers"]
import sys
for path in package_paths:
    sys.path.append(path)
import goalscorer_package.constants as c
import goalscorer_package.data_cleaning as dc
import goalscorer_package.modelling as m
import datetime as dt
import pandas as pd
import numpy as np
# import pymc as pm
# print(f"Running on PyMC v{pm.__version__}")
# import arviz as az
# print(f"Running on Az v{az.__version__}")
# import pytensor.tensor as pt
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import glob
from statistics import mode
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_colwidth", 100)
pd.options.display.float_format = "{: ,.3f}".format
%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 6)
%config InlineBackend.figure_formats = ["retina"]

# Data

In [2]:
seasons = ["2017-2018"]
comp_ids = [9, 10, 11, 12, 13, 20]

In [10]:
def data(seasons: list[str], comp_ids: list[int]) -> pd.DataFrame:
    seasons_leagues = dc.get_seasons_leagues_from_str(seasons, comp_ids)
    df_exp = dc.load_match_expectancies_df(seasons_leagues)
    df = dc.load_seasons_leagues_files("summary", True, seasons_leagues)
    df = dc.split_positions(df)
    df = dc.position_to_generic_position(df)
    df = dc.add_frac_90(df)
    df = dc.add_home(df)
    df = dc.add_opp_team(df)
    df = dc.drop_gk(df)
    df = dc.drop_na_npxg(df)
    df = dc.drop_na_frac_90(df)
    df = dc.add_league(df)
    return df

In [11]:
df = data(seasons, comp_ids)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\benja\\Documents\\projects\\goalscorers\\data\\files\\edited\\football-data\\\\2017-2018-league-9-historic-odds.csv'

In [ ]:
df_exp = dc.load_match_expectancies_df(seasons_leagues)
df_exp

In [ ]:
df

In [ ]:
df_model = df[["npxg", "player_id", "home", "opposition_team", "frac_90", "start", "position", "league", "squad"]].copy()
df_model.head()

In [21]:
(pd.to_datetime(df.datetime).dt.time)

0        00:00:01.502477
1        00:00:01.502477
2        00:00:01.502477
3        00:00:01.502477
4        00:00:01.502477
              ...       
46920    00:00:01.526131
46921    00:00:01.526131
46922    00:00:01.526131
46923    00:00:01.526131
46924    00:00:01.526131
Name: datetime, Length: 46925, dtype: object

In [23]:
print(dt.time(0, 0, 0, 0))

00:00:00
